In [19]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler  
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error
import numpy as np

In [20]:
df_smiles_origin = pd.read_csv('../../data/CCLE-GDSC-SMILES.csv')

In [21]:
from pytoda.smiles import SMILESTokenizer
smiles_language_filepath = '../../data/smiles_language/tokenizer_customized'
# Load SMILES language
smiles_language = SMILESTokenizer.from_pretrained(smiles_language_filepath)
smiles_language.set_encoding_transforms(
    add_start_and_stop=True,
    padding=True,
    padding_length=256,
    # padding_length=params.get("smiles_padding_length", None),
)
smiles_language.set_smiles_transforms(
    augment=False,
    canonical=False,
    kekulize=False,
    all_bonds_explicit=False,
    all_hs_explicit=False,
    remove_bonddir=False,
    remove_chirality=False,
    selfies=False,
    sanitize=False,
)
smiles_language.add_dataset(df_smiles_origin['SMILES'])

In [22]:
smiles = df_smiles_origin['SMILES'].values


In [23]:
# 将smiles全都转换为token
smiles_num_array = []
for smile in smiles:
    single_drug = smiles_language.smiles_to_token_indexes(smile)
    single_drug_num_array = np.array(single_drug)
    smiles_num_array.append(single_drug_num_array)
# 将smiles_num_array转换为dataframe
df_smiles = pd.DataFrame(smiles_num_array)

In [24]:
# 将df_smiles_origin的DRUG_NAME列插入到df_smiles的第一列
df_smiles.insert(0, 'drug', df_smiles_origin['DRUG_NAME'])
df_smiles.head()

,drug,0,1,2,3,4,5,6,7,8,...,246,247,248,249,250,251,252,253,254,255
0,Camptothecin,0,0,0,0,0,0,0,0,0,...,38,9,38,8,37,38,7,5,35,3
1,Vinblastine,0,0,0,0,0,0,0,0,0,...,38,4,37,35,5,35,38,5,35,3
2,Cisplatin,0,0,0,0,0,0,0,0,0,...,0,2,36,41,36,41,40,42,40,3
3,Cytarabine,0,0,0,0,0,0,0,0,0,...,35,7,5,38,35,5,35,5,35,3
4,Docetaxel,0,0,0,0,0,0,0,0,0,...,35,5,38,5,35,5,38,5,35,3


In [25]:
df_drug_sensitivity = pd.read_csv('../../data/drug_sensitivity.csv')
df_drug_sensitivity.head()

,drug,cell_line,IC50
0,5-Fluorouracil,HL60,2.558926
1,5-azacytidine,HL60,0.917132
2,A-366,HL60,4.836160
3,ABT737,HL60,-2.817798
4,AGI-5198,HL60,3.644734


In [26]:
# GEP
df_GEP = pd.read_csv('../data/GeneExp_Wilcoxon_test_Analysis_Log10_P_value_C2_KEGG_MEDICUS.csv')
df_GEP.head()
# CNV
# df_CNV = pd.read_csv('../data/CNV_Cardinality_analysis_of_variance_Latest_MEDICUS.csv')
# MUT
# df_MUT = pd.read_csv('../data/MUT_cardinality_analysis_of_variance_Only_MEDICUS.csv')

,cell_line,KEGG_MEDICUS_ENV_FACTOR_ARSENIC_TO_ELECTRON_TRANSFER_IN_COMPLEX_IV,KEGG_MEDICUS_ENV_FACTOR_BENZO_A_PYRENRE_TO_CYP_MEDIATED_METABOLISM,KEGG_MEDICUS_ENV_FACTOR_BPA_TO_RAS_ERK_SIGNALING_PATHWAY,KEGG_MEDICUS_ENV_FACTOR_DCE_TO_DNA_ADDUCTS,KEGG_MEDICUS_ENV_FACTOR_E2_TO_NUCLEAR_INITIATED_ESTROGEN_SIGNALING_PATHWAY,KEGG_MEDICUS_ENV_FACTOR_E2_TO_RAS_ERK_SIGNALING_PATHWAY,KEGG_MEDICUS_ENV_FACTOR_IRON_TO_ANTEROGRADE_AXONAL_TRANSPORT,KEGG_MEDICUS_ENV_FACTOR_METALS_TO_JNK_SIGNALING_PATHWAY,KEGG_MEDICUS_ENV_FACTOR_METALS_TO_KEAP1_NRF2_SIGNALIG_PATHWAY,...,KEGG_MEDICUS_VARIANT_SCRAPIE_CONFORMATION_PRPSC_TO_26S_PROTEASOME_MEDIATED_PROTEIN_DEGRADATION,KEGG_MEDICUS_VARIANT_SCRAPIE_CONFORMATION_PRPSC_TO_MGLUR5_CA2_APOPTOTIC_PATHWAY,KEGG_MEDICUS_VARIANT_SCRAPIE_CONFORMATION_PRPSC_TO_PERK_ATF4_SIGNALING_PATHWAY,KEGG_MEDICUS_VARIANT_SCRAPIE_CONFORMATION_PRPSC_TO_PRNP_PI3K_NOX2_SIGNALING_PATHWAY,KEGG_MEDICUS_VARIANT_SCRAPIE_CONFORMATION_PRPSC_TO_TRANSPORT_OF_CALCIUM,KEGG_MEDICUS_VARIANT_SCRAPIE_CONFORMATION_PRPSC_TO_VGCC_CA2_APOPTOTIC_PATHWAY,KEGG_MEDICUS_VARIANT_TEL_AML1_FUSION_TO_TRANSCRIPTIONAL_REPRESSION,KEGG_MEDICUS_VARIANT_TGFA_OVEREXPRESSION_TO_PI3K_SIGNALING_PATHWAY,KEGG_MEDICUS_VARIANT_TMPRSS2_ERG_FUSION_TO_TRANSCRIPTIONAL_ACTIVATION,KEGG_MEDICUS_VARIANT_TRK_FUSION_KINASE_TO_RAS_ERK_SIGNALING_PATHWAY
0,HL60,4.214134,0.440202,1.470915,0.205382,1.450508,1.482270,0.343427,0.748859,0.372432,...,17.685880,1.689896,2.603777,3.056268,1.462969,1.259628,1.490945,0.952237,0.182561,1.939787
1,HEL,3.646497,0.135204,1.566388,0.925705,1.111581,0.883885,0.949699,0.261281,0.576484,...,15.719172,1.853080,1.901783,1.311271,0.849546,0.863047,0.123552,0.346157,1.325299,2.701512
2,MONOMAC6,4.472972,0.014476,1.661451,0.019613,1.938533,1.503992,0.675432,0.944259,0.700977,...,17.283634,1.538003,3.062976,2.903670,1.140496,0.942227,0.708988,0.904325,0.409584,2.088677
3,LS513,4.352599,0.462321,1.699385,0.407889,0.665094,2.222235,0.398729,1.351826,1.652013,...,16.863173,1.824951,2.086926,0.256067,1.761924,0.436037,0.022385,0.619383,0.759916,1.872848
4,A101D,4.053421,0.062876,1.848867,0.094523,1.103970,1.862741,2.710302,0.520589,1.123209,...,19.239258,2.301589,3.182736,0.921441,1.438838,0.630094,0.104327,1.590609,0.198160,2.121071


In [27]:
# 将df_drug_sensitivity作为主要的dataframe，然后将df_smiles和df_GEP合并到df_drug_sensitivity
df = pd.merge(df_drug_sensitivity, df_GEP, on='cell_line')
df = pd.merge(df, df_smiles , on='drug')
print(df.shape)
df.head()

(141222, 878)


,drug,cell_line,IC50,KEGG_MEDICUS_ENV_FACTOR_ARSENIC_TO_ELECTRON_TRANSFER_IN_COMPLEX_IV,KEGG_MEDICUS_ENV_FACTOR_BENZO_A_PYRENRE_TO_CYP_MEDIATED_METABOLISM,KEGG_MEDICUS_ENV_FACTOR_BPA_TO_RAS_ERK_SIGNALING_PATHWAY,KEGG_MEDICUS_ENV_FACTOR_DCE_TO_DNA_ADDUCTS,KEGG_MEDICUS_ENV_FACTOR_E2_TO_NUCLEAR_INITIATED_ESTROGEN_SIGNALING_PATHWAY,KEGG_MEDICUS_ENV_FACTOR_E2_TO_RAS_ERK_SIGNALING_PATHWAY,KEGG_MEDICUS_ENV_FACTOR_IRON_TO_ANTEROGRADE_AXONAL_TRANSPORT,...,246,247,248,249,250,251,252,253,254,255
0,5-Fluorouracil,HL60,2.558926,4.214134,0.440202,1.470915,0.205382,1.450508,1.48227,0.343427,...,38,4,37,35,5,36,6,5,43,3
1,5-azacytidine,HL60,0.917132,4.214134,0.440202,1.470915,0.205382,1.450508,1.48227,0.343427,...,5,38,35,5,35,5,35,5,36,3
2,A-366,HL60,4.836160,4.214134,0.440202,1.470915,0.205382,1.450508,1.48227,0.343427,...,38,38,36,9,38,38,38,38,9,3
3,ABT737,HL60,-2.817798,4.214134,0.440202,1.470915,0.205382,1.450508,1.48227,0.343427,...,5,40,5,50,4,37,35,5,49,3
4,AGI-5198,HL60,3.644734,4.214134,0.440202,1.470915,0.205382,1.450508,1.48227,0.343427,...,9,38,37,38,36,37,38,9,38,3


In [28]:
# df取1000条数据
df = df.head(3000)

In [29]:
# 将每个cell_line的数据9：1划分为训练集和测试集
df_train = pd.DataFrame()
df_test = pd.DataFrame()
for cell_line in df['cell_line'].unique():
    df_cell_line = df[df['cell_line'] == cell_line]
    df_cell_line_train, df_cell_line_test = train_test_split(df_cell_line, test_size=0.1, random_state=42)
    df_train = pd.concat([df_train, df_cell_line_train])
    df_test = pd.concat([df_test, df_cell_line_test])

In [30]:
X_train,X_test,y_train,y_test = df_train.iloc[:, 3:], df_test.iloc[:, 3:], df_train['IC50'], df_test['IC50'] 
X_train.columns = range(X_train.shape[1])
X_test.columns = range(X_test.shape[1])

In [31]:
# RF 模型
rf = RandomForestRegressor()
rf.fit(X_train, y_train)
rf_pred = rf.predict(X_test)
rf_mse = mean_squared_error(y_test, rf_pred)
# 计算R2
rf_r2 = rf.score(X_test, y_test)
# 计算皮尔逊相关系数
rf_pearson = np.corrcoef(rf_pred, y_test)[0, 1]
# 将y_test和rf_pred转换为dataframe，打印成csv
df_rf = pd.DataFrame({'y_test': y_test, 'rf_pred': rf_pred})
df_rf.to_csv('rf.csv')

In [32]:
# SVM 模型
svm = SVR()
svm.fit(X_train, y_train)
svm_pred = svm.predict(X_test)
svm_mse = mean_squared_error(y_test, svm_pred)
# 计算R2
svm_r2 = svm.score(X_test, y_test)
# 计算皮尔逊相关系数
svm_pearson = np.corrcoef(svm_pred, y_test)[0, 1]
# 将y_test和svm_pred转换为dataframe，打印成csv
df_svm = pd.DataFrame({'y_test': y_test, 'svm_pred': svm_pred})
df_svm.to_csv('svm.csv')

In [33]:
print('RF MSE:', rf_mse)
print('RF R2:', rf_r2)
print('RF Pearson:', rf_pearson)

print('SVM MSE:', svm_mse)
print('SVM R2:', svm_r2)
print('SVM Pearson:', svm_pearson)

RF MSE: 2.0108095615283377
RF R2: 0.7387342093644378
RF Pearson: 0.8600558565025528
SVM MSE: 4.519762705672764
SVM R2: 0.4127442999200791
SVM Pearson: 0.6787064400755944


In [ ]:
# ========================下面为随机分配数据，上面是严格的MIXED SET===========================

In [10]:
# df取1000条数据
# df = df.head(1000)

In [11]:
# 分割特征X：df从第三列开始，y：IC50列
X = df.iloc[:, 3:]
# 将X的columns重置为0,1,2,3...
X.columns = range(X.shape[1])
y = df['IC50']

In [12]:
# 划分训练集测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

In [13]:
# 特征工程
# scaler = StandardScaler().fit(X_train)
# X_train_scaled = scaler.transform(X_train)
# X_test_scaled = scaler.transform(X_test)

In [13]:
# RF 模型
rf = RandomForestRegressor()
rf.fit(X_train, y_train)
rf_pred = rf.predict(X_test)
rf_mse = mean_squared_error(y_test, rf_pred)
# 计算R2
rf_r2 = rf.score(X_test, y_test)
# 计算皮尔逊相关系数
rf_pearson = np.corrcoef(rf_pred, y_test)[0, 1]
# 将y_test和rf_pred转换为dataframe，打印成csv
df_rf = pd.DataFrame({'y_test': y_test, 'rf_pred': rf_pred})
df_rf.to_csv('rf.csv')

In [14]:
# SVM 模型
svm = SVR()
svm.fit(X_train, y_train)
svm_pred = svm.predict(X_test)
svm_mse = mean_squared_error(y_test, svm_pred)
# 计算R2
svm_r2 = svm.score(X_test, y_test)
# 计算皮尔逊相关系数
svm_pearson = np.corrcoef(svm_pred, y_test)[0, 1]
# 将y_test和svm_pred转换为dataframe，打印成csv
df_svm = pd.DataFrame({'y_test': y_test, 'svm_pred': svm_pred})
df_svm.to_csv('svm.csv')

In [15]:
print('RF MSE:', rf_mse)
print('RF R2:', rf_r2)
print('RF Pearson:', rf_pearson)

print('SVM MSE:', svm_mse)
print('SVM R2:', svm_r2)
print('SVM Pearson:', svm_pearson)

RF MSE: 1.809792379981703
RF R2: 0.7243196198297389
RF Pearson: 0.851243231995943
SVM MSE: 4.97780528415696
SVM R2: 0.24174547957606118
SVM Pearson: 0.5163799726994603
